# Lab 2 - Assignment

In [1]:
from datetime import date
today = date.today()
d2 = today.strftime("%B %d, %Y")
print("Updated by Dylan McPeake, ", d2)

Updated by Dylan McPeake,  February 22, 2024


In [2]:
import ee
ee.Initialize()

import geemap

Map = geemap.Map() 
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [3]:
latitude1 = 41.754372
longitude1 = -111.834412
sfoPoint = ee.Geometry.Point([longitude1, latitude1]);
latitude2 =37.77862975352838
longitude2 = -122.22365888039644
sfoPoint2 = ee.Geometry.Point([longitude2, latitude2]);

# Modis

In [4]:
myd09 = ee.ImageCollection("MODIS/061/MCD43A4")

# Get a surface reflectance image from the MODIS MYD09GA collection.
modisImage = ee.Image(myd09.filterDate('2022-07-05').first());

modisImage = modisImage.multiply(0.0001)
# Print the image band names to the console.
band_names = modisImage.bandNames()
print(band_names.getInfo())

['Nadir_Reflectance_Band1', 'Nadir_Reflectance_Band2', 'Nadir_Reflectance_Band3', 'Nadir_Reflectance_Band4', 'Nadir_Reflectance_Band5', 'Nadir_Reflectance_Band6', 'Nadir_Reflectance_Band7', 'BRDF_Albedo_Band_Mandatory_Quality_Band1', 'BRDF_Albedo_Band_Mandatory_Quality_Band2', 'BRDF_Albedo_Band_Mandatory_Quality_Band3', 'BRDF_Albedo_Band_Mandatory_Quality_Band4', 'BRDF_Albedo_Band_Mandatory_Quality_Band5', 'BRDF_Albedo_Band_Mandatory_Quality_Band6', 'BRDF_Albedo_Band_Mandatory_Quality_Band7']


In [5]:
# Use these MODIS bands for red, green, blue, respectively.
modisBands = ['Nadir_Reflectance_Band1', 'Nadir_Reflectance_Band4', 'Nadir_Reflectance_Band3'];

#// Add the image to the map.
# url = modisImage.getThumbURL({'dimensions': 700,'bands':modisBands,'min': 0, 'max': 3000,'region':area_to_display})
# # print(url)
# Image(url=url)

# add layer
Map.addLayer(modisImage, {'bands':modisBands,'min': 0, 'max':0.4}, name='MODIS Image')
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [6]:
# Get the scale of the data from the first band's projection:
modisScale = modisImage.select('Nadir_Reflectance_Band1').projection().nominalScale();

print('MODIS scale:', modisScale.getInfo());

MODIS scale: 463.31271652791656


# Landsat MSS

In [7]:
# Filter MSS imagery by location, date and cloudiness.
mss= ee.ImageCollection("LANDSAT/LM01/C02/T1")

mssImage = ee.Image(mss
    .filterBounds(sfoPoint2)
    #.filterDate('1972-07-26','1978-01-06')
    .sort('CLOUD_COVER')
#Get the least cloudy image.
    .first());




# Print the image band names to the console.
band_names = mssImage.bandNames()
print(band_names.getInfo())
# Display the MSS image as a color-IR composite.

['B4', 'B5', 'B6', 'B7', 'QA_PIXEL', 'QA_RADSAT']


In [8]:
# Use these MODIS bands for red, green, blue, respectively.
mssBands = ['B6', 'B5', 'B4'];

Map.addLayer(mssImage, {'bands':mssBands,'min': 0, 'max': 70}, name='LandsatMSS Image')
Map
# bounds = sfoPoint.buffer(70000)
# Map.centerObject(bounds)

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [9]:
# Get the scale of the MSS data from its projection:
mssScale = mssImage.select('B6').projection().nominalScale();

print('MSS scale:', mssScale.getInfo());


MSS scale: 60


There is no LANDSAT 1 data for Logan, Utah. San Fransicso, Californi was used here to show how importation and work and the scale.

# TM

In [10]:
tm= ee.ImageCollection("LANDSAT/LT05/C02/T1_L2")

# // Filter TM imagery by location, date and cloudiness.
tmImage = ee.Image(tm
    .filterBounds(sfoPoint) # I could have used sfopoint too. every image has a boundary geometry
    .filterDate('2011-06-01', '2011-07-01')
    .sort('CLOUD_COVER') # not all satellites have this property, check the properties table
    .first());

tmImage = tmImage.multiply(2.75e-05).add(-0.2)


# Print the image band names to the console.
band_names = tmImage.bandNames()
print(band_names.getInfo())

['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B7', 'SR_ATMOS_OPACITY', 'SR_CLOUD_QA', 'ST_B6', 'ST_ATRAN', 'ST_CDIST', 'ST_DRAD', 'ST_EMIS', 'ST_EMSD', 'ST_QA', 'ST_TRAD', 'ST_URAD', 'QA_PIXEL', 'QA_RADSAT']


In [11]:
# // Display the TM image as a color-IR composite.
tmBands=  ['SR_B3', 'SR_B2', 'SR_B1']

Map.addLayer(tmImage, {'bands':tmBands,'min': 0, 'max': 0.4}, name='Landsat TM Image')
Map
# bounds = sfoPoint.buffer(70000)
# Map.centerObject(bounds)

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [12]:
# // Get the scale of the TM data from its projection:
tmScale = tmImage.select('SR_B1').projection().nominalScale();
print('TM scale:', tmScale.getInfo());

TM scale: 30


# NAIP

In [13]:
naip = ee.ImageCollection("USDA/NAIP/DOQQ")

# // Get NAIP images for the study period and region of interest.
naipImages = naip.filterDate('2019-01-01', '2023-12-31').filterBounds(sfoPoint);

# // Mosaic adjacent images into a single image.
naipImage = naipImages.mosaic();

# Print the image band names to the console.
band_names = naipImage.bandNames()
print(band_names.getInfo())

['R', 'G', 'B', 'N']


In [14]:
# // Display the NAIP mosaic as a color-IR composite.
nBands=  ['R', 'G', 'B']

Map.addLayer(naipImage, {'bands':nBands,'min': 0, 'max': 255}, name='NAIP Image')
Map
# bounds = sfoPoint.buffer(70000)
# Map.centerObject(bounds)

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [15]:
# // Get the NAIP resolution from the first image in the mosaic.
naipScale = ee.Image(naipImages.first()).projection().nominalScale();
print('NAIP scale:', naipScale.getInfo());


NAIP scale: 0.6


# Aster

In [16]:
aster= ee.ImageCollection("ASTER/AST_L1T_003")

asterImages = ee.Image(aster
    .filterBounds(sfoPoint)
    .filterDate('2020-01-01','2021-12-31')
    .sort('CLOUDCOVER')
    .first());

# Print the image band names to the console.
band_names = asterImages.bandNames()
print(band_names.getInfo())                 

['B01', 'B02', 'B3N', 'B04', 'B05', 'B06', 'B07', 'B08', 'B09', 'B10', 'B11', 'B12', 'B13', 'B14']


In [17]:
# // Display the TM image as a color-IR composite.
asterBands=  ['B3N', 'B02', 'B01']

Map.addLayer(asterImages, {'bands':asterBands,'min': 0, 'max': 255}, name='Aster Image')
Map
# bounds = sfoPoint.buffer(70000)
# Map.centerObject(bounds)

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [18]:
# Get the scale of the MSS data from its projection:
asterScale = asterImages.select('B01').projection().nominalScale();

print('Aster scale:', asterScale.getInfo());

Aster scale: 15


# Sentinel 2

In [19]:
sentinel2= ee.ImageCollection("COPERNICUS/S2")

sentinel2Image= ee.Image(sentinel2
    .filterBounds(sfoPoint)
    .filterDate('2023-01-01','2023-12-31')
    .sort('CLOUDY_PIXEL_PERCENTAGE')
    .first());

sentinel2Image = sentinel2Image.multiply(0.0001)

band_names = sentinel2Image.bandNames()
print(band_names.getInfo())

['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B10', 'B11', 'B12', 'QA10', 'QA20', 'QA60']


In [20]:
sentinel2Bands=  ['B4', 'B3', 'B2']

Map.addLayer(sentinel2Image, {'bands':sentinel2Bands,'min': 0, 'max': 1}, name='Sentinel 2 Image')
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [21]:
sentinel2Scale = sentinel2Image.select('B1').projection().nominalScale();
print('Sentinel 2 scale:', sentinel2Scale.getInfo());

Sentinel 2 scale: 60


# Sentinel 3

In [22]:
sentinel3= ee.ImageCollection("COPERNICUS/S3/OLCI")

sentinel3Image= ee.Image(sentinel3
    .filterBounds(sfoPoint)
    .filterDate('2023-01-01','2023-12-31')
    .first());

#EACH BAND HAS DIFFERENT SCALES

band_names = sentinel3Image.bandNames()
print(band_names.getInfo())

['Oa01_radiance', 'Oa02_radiance', 'Oa03_radiance', 'Oa04_radiance', 'Oa05_radiance', 'Oa06_radiance', 'Oa07_radiance', 'Oa08_radiance', 'Oa09_radiance', 'Oa10_radiance', 'Oa11_radiance', 'Oa12_radiance', 'Oa13_radiance', 'Oa14_radiance', 'Oa15_radiance', 'Oa16_radiance', 'Oa17_radiance', 'Oa18_radiance', 'Oa19_radiance', 'Oa20_radiance', 'Oa21_radiance', 'quality_flags']


In [23]:
sentinel3Bands=  ['Oa08_radiance', 'Oa06_radiance', 'Oa03_radiance']

Map.addLayer(sentinel3Image, {'bands':sentinel3Bands,'min': 0, 'max': 255}, name='Sentinel 3 Image')
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [24]:
sentinel3Scale = sentinel3Image.select('Oa01_radiance').projection().nominalScale();
print('Sentinel 3 scale:', sentinel3Scale.getInfo());

Sentinel 3 scale: 310.11803522705475


# Proba-V

In [25]:
proba= ee.ImageCollection("VITO/PROBAV/C1/S1_TOC_100M")

probaImage= ee.Image(proba
    .filterBounds(sfoPoint)
    .filterDate('2019-01-01','2019-12-31')
    .sort('CLOUD_COVER_PERCENTAGE')
    .first());

band_names = probaImage.bandNames()
print(band_names.getInfo())

['RED', 'NIR', 'BLUE', 'SWIR', 'NDVI', 'SZA', 'SAA', 'SWIRVAA', 'SWIRVZA', 'VNIRVAA', 'VNIRVZA', 'SM', 'TIME']


In [26]:
probaBands=  ['NIR', 'RED', 'BLUE']

Map.addLayer(probaImage, {'bands':probaBands,'min': 0, 'max': 1000}, name='Proba-V Image')
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [27]:
probaScale = probaImage.select('RED').projection().nominalScale();
print('Proba-V scale:', probaScale.getInfo());

Proba-V scale: 110.43600277104996


# VIIRS

In [28]:
viirs= ee.ImageCollection("NOAA/VIIRS/001/VNP09GA")

viirsImage= ee.Image(viirs
    .filterBounds(sfoPoint)
    .filterDate('2023-01-01','2023-12-31')
    .first());

viirsImage = viirsImage.multiply(0.0001)

band_names = viirsImage.bandNames()
print(band_names.getInfo())

['I1', 'I2', 'I3', 'M1', 'M2', 'M3', 'M4', 'M5', 'M7', 'M8', 'M10', 'M11', 'SensorAzimuth', 'SensorZenith', 'SolarAzimuth', 'SolarZenith', 'iobs_res', 'num_observations_1km', 'num_observations_500m', 'obscov_1km', 'obscov_500m', 'orbit_pnt', 'QF1', 'QF2', 'QF3', 'QF4', 'QF5', 'QF6', 'QF7']


In [29]:
viirsBands=  ['M5', 'M4', 'M3']

Map.addLayer(viirsImage, {'bands':viirsBands,'min': 0, 'max': 1}, name='VIIRS Image')
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [30]:
viirsScale = viirsImage.select('M5').projection().nominalScale();
print('VIIRS scale:', viirsScale.getInfo());

VIIRS scale: 926.625433055833


# HYPERSPECTRAL SENSOR EO-1

In [31]:
hyper= ee.ImageCollection("EO1/HYPERION")

hyperImage= ee.Image(hyper
    .filterBounds(sfoPoint)
    .filterDate('2001-05-01','2017-03-13')
    .first());

band_names = hyperImage.bandNames()
print(band_names.getInfo())

['B008', 'B009', 'B010', 'B011', 'B012', 'B013', 'B014', 'B015', 'B016', 'B017', 'B018', 'B019', 'B020', 'B021', 'B022', 'B023', 'B024', 'B025', 'B026', 'B027', 'B028', 'B029', 'B030', 'B031', 'B032', 'B033', 'B034', 'B035', 'B036', 'B037', 'B038', 'B039', 'B040', 'B041', 'B042', 'B043', 'B044', 'B045', 'B046', 'B047', 'B048', 'B049', 'B050', 'B051', 'B052', 'B053', 'B054', 'B055', 'B056', 'B057', 'B077', 'B078', 'B079', 'B080', 'B081', 'B082', 'B083', 'B084', 'B085', 'B086', 'B087', 'B088', 'B089', 'B090', 'B091', 'B092', 'B093', 'B094', 'B095', 'B096', 'B097', 'B098', 'B099', 'B100', 'B101', 'B102', 'B103', 'B104', 'B105', 'B106', 'B107', 'B108', 'B109', 'B110', 'B111', 'B112', 'B113', 'B114', 'B115', 'B116', 'B117', 'B118', 'B119', 'B120', 'B121', 'B122', 'B123', 'B124', 'B125', 'B126', 'B127', 'B128', 'B129', 'B130', 'B131', 'B132', 'B133', 'B134', 'B135', 'B136', 'B137', 'B138', 'B139', 'B140', 'B141', 'B142', 'B143', 'B144', 'B145', 'B146', 'B147', 'B148', 'B149', 'B150', 'B151',

In [32]:
hyperBands=  ['B030', 'B020', 'B011']

Map.addLayer(hyperImage, {'bands':hyperBands,'min': 0, 'max': 5000}, name='HYPERSPECTRAL SENSOR EO-1 Image')
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [33]:
hyperScale = hyperImage.select('B030').projection().nominalScale();
print('HYPERSPECTRAL SENSOR EO-1 scale:', hyperScale.getInfo());

HYPERSPECTRAL SENSOR EO-1 scale: 29.989829492111358
